In [ ]:
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
import numpy as np

# Carregar o arquivo Excel
file_path = r"C:\Users\Conrado\Desktop\A1xA2_All.xlsx"
df = pd.read_excel(file_path)

# Verificar se a coluna Description existe
description_col = "Description"
if description_col not in df.columns:
    raise ValueError(f"A coluna '{description_col}' não foi encontrada no arquivo Excel.")

# Definir colunas dos grupos com os novos nomes
grupo_CTRL = ["CTRL_1", "CTRL_2", "CTRL_3"]
grupo_TRT = ["TRT_1", "TRT_2", "TRT_3"]

# Verificar se todas as colunas estão presentes
for col in grupo_CTRL + grupo_TRT:
    if col not in df.columns:
        raise ValueError(f"A coluna '{col}' não foi encontrada no arquivo Excel.")

# Converter colunas numéricas para float
df[grupo_CTRL + grupo_TRT] = df[grupo_CTRL + grupo_TRT].apply(pd.to_numeric, errors='coerce')

# Transformar os valores para log2 (evitando valores negativos ou zero)
df[grupo_CTRL + grupo_TRT] = df[grupo_CTRL + grupo_TRT].applymap(lambda x: np.log2(x) if x > 0 else np.nan)

# Inicializar listas para armazenar resultados
t_values = []
p_values = []
means_CTRL = []
means_TRT = []
stds_CTRL = []
stds_TRT = []
n_CTRL_list = []  # Usar uma lista para armazenar os números de amostras
n_TRT_list = []  # Usar uma lista para armazenar os números de amostras

# Realizar o teste t para cada linha (proteína)
for _, row in df.iterrows():
    valores_CTRL = row[grupo_CTRL].dropna().astype(float)
    valores_TRT = row[grupo_TRT].dropna().astype(float)
    
    # Calcular as estatísticas para cada grupo
    mean_CTRL = valores_CTRL.mean() if len(valores_CTRL) > 0 else np.nan
    mean_TRT = valores_TRT.mean() if len(valores_TRT) > 0 else np.nan
    std_CTRL = valores_CTRL.std() if len(valores_CTRL) > 0 else np.nan
    std_TRT = valores_TRT.std() if len(valores_TRT) > 0 else np.nan
    n_CTRL = len(valores_CTRL)
    n_TRT = len(valores_TRT)
    
    # Realizar o teste t, se houver pelo menos 2 valores em cada grupo
    if n_CTRL > 1 and n_TRT > 1:
        t_stat, p_val = stats.ttest_ind(valores_CTRL, valores_TRT, nan_policy='omit')
    else:
        t_stat, p_val = None, None  # Se não houver dados suficientes, deixa como None

    # Armazenar os resultados
    t_values.append(t_stat)
    p_values.append(p_val)
    means_CTRL.append(mean_CTRL)
    means_TRT.append(mean_TRT)
    stds_CTRL.append(std_CTRL)
    stds_TRT.append(std_TRT)
    n_CTRL_list.append(n_CTRL)  # Adicionando os valores corretamente à lista
    n_TRT_list.append(n_TRT)  # Adicionando os valores corretamente à lista

# Ajuste para múltiplos testes (Benjamini-Hochberg)
p_values_clean = [p for p in p_values if p is not None]  
p_adjusted = multipletests(p_values_clean, method='fdr_bh')[1] if p_values_clean else [None] * len(p_values)

# Criar um DataFrame com os resultados
results = df[["Description"]].copy()  # Mantém a coluna Description
results["Média Grupo CTRL (log2)"] = means_CTRL
results["Média Grupo TRT (log2)"] = means_TRT
results["Desvio Padrão CTRL"] = stds_CTRL
results["Desvio Padrão TRT"] = stds_TRT
results["N Amostras CTRL"] = n_CTRL_list  # Usando a lista corrigida
results["N Amostras TRT"] = n_TRT_list  # Usando a lista corrigida
results["t-valor"] = t_values
results["p-valor"] = p_values
results["p-ajustado"] = pd.NA  # Inicializa com NaN
results.loc[results["p-valor"].notna(), "p-ajustado"] = p_adjusted  # Apenas onde há p-valor válido

# Salvar os resultados em um novo arquivo Excel
output_path = r"C:\Users\Conrado\Desktop\Resultados_TTest_Detalhado.xlsx"
results.to_excel(output_path, index=False)

print(f"Análise concluída! Resultados salvos em: {output_path}")


Análise concluída! Resultados salvos em: C:\Users\Conrado\Desktop\Resultados_TTest_Detalhado.xlsx
